### Obteniendo coordenadas de direcciones físicas con la API de google Geocoding

Es posible aprovechar la API de google Geocoding para obtener las coordenada (latitud y longitud) de una dirección. Por ejemplo si queremos obtener las coordenada de CARRERA 81C CALLE 4C-61, CARTAGENA, BOLIVAR para poder colocarlas en un mapa para visualización. 

Es muy frecuente que tengamos miles de direcciones para encontrar sus coordenadas y hacerlo manuelamente es un atarea tediosa. Por lo tanto utilizar la API de google puede ser de gran ayuda.

Librerias a utilizar

In [1]:
import pandas as pd
import numpy as np
import googlemaps
import re

In [8]:
## funciones de limpieza de las direcciones


def recodificarCalle(direccion):
    expresion="[C][L][\s*]|[C][L][L][E][\s*]|[C][L][L][\s*]|[C][l][l][\s*]"
    recod=re.sub(expresion,"CALLE ",direccion)
    return recod

def recodificarCarrera(direccion):
    expresion="[C][R][\s*]|[K][R][A][\s*]|[K][A][\s*]|[C][A][\s*]|[C][R][A][\s*]|[C][r][a][\s*]|[K][R][\s*]"
    recod=re.sub(expresion,"CARRERA ",direccion)
    return recod

def recodificarLocal(direccion):
    expresion="[L][C][\s*]|[l][c][\s*]|[L][C][.][\s*]"
    recod=re.sub(expresion,"LOCAL ",direccion)
    return recod
    
def recodificarAvenida(direccion):
    expresion="[A][V][\s*]|[a][v][\s*]"
    recod=re.sub(expresion,"AVENIDA ",direccion)
    return recod
	
def recodificarDiagonal(direccion):
    expresion="[D][G][\s*]|[d][g][\s*]|[D][G][.][\s*]"
    recod=re.sub(expresion,"DIAGONAL ",direccion)
    return recod
	
def recodificarManzana(direccion):
    expresion="[M][Z][\s*]|[m][z][\s*]"
    recod=re.sub(expresion,"MANZANA ",direccion)
    return recod
	
def recodificarVereda(direccion):
    expresion="[V][R][D][\s*]|[m][z][\s*]"
    recod=re.sub(expresion,"VEREDA ",direccion)
    return recod

def limpiarTexto(texto):
    texto_limpio=texto.lstrip().rstrip()
    return texto_limpio

### funcion para obtener la latitud y la longitud, el try except es necesario para no interrumpir la ejecución en caso de no encontrar coordenadas para la direccion.
def obtenerCoordenadas(add):
    g = key_google.geocode(add)
    try:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    except IndexError:
        lat='null'
        lng='null'

# Funcion consolidada para limpieza previa de la direccion.
def procesarDireccion(direccion):
    dir_clean=limpiarTexto(direccion)
    dir_mod=recodificarVereda(recodificarManzana(recodificarDiagonal(
        recodificarAvenida(recodificarLocal(recodificarCarrera(
                    recodificarCalle(dir_clean))
                    )))))
    return dir_mod


La primer paso para consumir la API de google es abrir una cuenta en Google Cloud, crear un proyecto y en este habilitar la API geocoding. URL de referencia:
https://developers.google.com/maps/documentation/geocoding/get-api-key

Una vez contamos con la API podemos hacer el llamado de la misma para poder utilziar la API:

In [3]:
key_google = googlemaps.Client(key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

## PRUEBAS

In [12]:
texto="CL 32 SUR 52 A 54, Bogota, Colombia"
texto_procesado=procesarDireccion(texto)

In [13]:
lista_dir=[texto_procesado]
df=pd.DataFrame({'direcciones':lista_dir})
df

,direcciones
0,"CALLE 32 SUR 52 A 54, Bogota, Colombia"


In [14]:
df['latitud-longitud'] = df['direcciones'].apply(obtenerCoordenadas)
df

,direcciones,latitud-longitud
0,"CALLE 32 SUR 52 A 54, Bogota, Colombia","(4.6050717, -74.12871609999999)"


## CONSULTA PARA TODA LA BASE

Datos abiertos: https://www.datos.gov.co/Educaci-n/LISTADO-COLEGIOS-BOGOTA/qijw-htwa

In [15]:
df=pd.read_csv("LISTADO_COLEGIOS_BOGOTA.csv",delimiter=",")
df.head()

,año,secretaria,nombreestablecimiento,zona,direccion,telefono,nombre_Rector,tipo_Establecimiento,etnias,sector,...,numero_de_Sedes,estado,prestador_de_Servicio,propiedad_Planta_Fisica,resguardo,matricula_Contratada,calendario,internado,estrato_Socio_Economico,correo_Electronico
0,2016,BOGOTA,INST DE BTO TEC COMERCIAL PITAGORAS ...,URBANA,KR 5 11 67,5665677 / 3429092,YOLANDA ARIAS CRUZ,INSTITUCION EDUCATIVA,NaN,NaN,...,1,NaN,PERSONA NATURAL,PERSONA NATURAL,NO APLICA,NO,A,NaN,NaN,pitagoras@pitagoras.edu.co
1,2016,BOGOTA,GIMNASIO INFANTIL FEDERICO FROEBEL,URBANA,CL 32 SUR 52 A 54,7100349,SANDRA MILENA MENDEZ,CENTRO EDUCATIVO,NaN,NaN,...,1,NaN,PERSONA NATURAL,PERSONA NATURAL,NO APLICA,NO,A,NaN,NaN,gimfedebel@hotmail.com
2,2016,BOGOTA,GIMNASIO INFANTIL LOS NIÑOS DEL MAÑANA,URBANA,CL 134 D 47 27,6010350,CLAUDIA BOCACHICA SAENZ,INSTITUCION EDUCATIVA,NaN,NaN,...,1,NaN,PERSONA NATURAL,PERSONA NATURAL,NO APLICA,NO,A,NaN,NaN,claudialindenibocachica@hotmail.com
3,2016,BOGOTA,COLEGIO FRANCISCO PRIMERO S.S. (IED),URBANA,KR 28 63 64,2126019 -- 2481271,JOSE CUSTODIO GALARZA MORENO,INSTITUCION EDUCATIVA,NaN,NaN,...,4,NaN,OFICIAL,OFICIAL,NO APLICA,NO,A,NaN,NaN,cedalemania12@redp.edu.co
4,2016,BOGOTA,COL SAN JUAN VIANETH ...,URBANA,KR 2 C 3 18 SUR,2804738 -- 4714934,GLORIA ESPERANZA CHUQUIN LAISECA,CENTRO EDUCATIVO,NaN,NaN,...,1,NaN,PERSONA NATURAL,PERSONA NATURAL,NO APLICA,NO,A,NaN,NaN,colegiosanjuanviateh@hotmail.com -- colegiosan...


In [16]:
seleccion_columnas=['nombreestablecimiento','direccion']
df2=df[seleccion_columnas]
df2['direccion-completa']=df2['direccion']+',Bogota'+',Colombia'
df2

C:\Users\1061708930\AppData\Local\Temp\ipykernel_14972\3631927309.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['direccion-completa']=df2['direccion']+',Bogota'+',Colombia'


,nombreestablecimiento,direccion,direccion-completa
0,INST DE BTO TEC COMERCIAL PITAGORAS ...,KR 5 11 67,"KR 5 11 67,Bogota,Colombia"
1,GIMNASIO INFANTIL FEDERICO FROEBEL,CL 32 SUR 52 A 54,"CL 32 SUR 52 A 54,Bogota,Colombia"
2,GIMNASIO INFANTIL LOS NIÑOS DEL MAÑANA,CL 134 D 47 27,"CL 134 D 47 27,Bogota,Colombia"
3,COLEGIO FRANCISCO PRIMERO S.S. (IED),KR 28 63 64,"KR 28 63 64,Bogota,Colombia"
4,COL SAN JUAN VIANETH ...,KR 2 C 3 18 SUR,"KR 2 C 3 18 SUR,Bogota,Colombia"
...,...,...,...
2399,COLEGIO SAN JOSE DE CALASANZ,CL 129 88 B 35,"CL 129 88 B 35,Bogota,Colombia"
2400,COL NUEVO REINO DE TURINGIA,KR 103 B 152 C 34,"KR 103 B 152 C 34,Bogota,Colombia"
2401,LIC INF GOOFY ...,CL 65 F SUR 78 C 32,"CL 65 F SUR 78 C 32,Bogota,Colombia"
2402,LIC APRENDO CON MIS AMIGOS ...,CR. 94 NO.29-30,"CR. 94 NO.29-30,Bogota,Colombia"


In [19]:
df2['direccion-corregida']=df2['direccion-completa'].apply(procesarDireccion)
df2

C:\Users\1061708930\AppData\Local\Temp\ipykernel_14972\4215807788.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['direccion-corregida']=df2['direccion-completa'].apply(procesarDireccion)


,nombreestablecimiento,direccion,direccion-completa,direccion-corregida
0,INST DE BTO TEC COMERCIAL PITAGORAS ...,KR 5 11 67,"KR 5 11 67,Bogota,Colombia","CARRERA 5 11 67,Bogota,Colombia"
1,GIMNASIO INFANTIL FEDERICO FROEBEL,CL 32 SUR 52 A 54,"CL 32 SUR 52 A 54,Bogota,Colombia","CALLE 32 SUR 52 A 54,Bogota,Colombia"
2,GIMNASIO INFANTIL LOS NIÑOS DEL MAÑANA,CL 134 D 47 27,"CL 134 D 47 27,Bogota,Colombia","CALLE 134 D 47 27,Bogota,Colombia"
3,COLEGIO FRANCISCO PRIMERO S.S. (IED),KR 28 63 64,"KR 28 63 64,Bogota,Colombia","CARRERA 28 63 64,Bogota,Colombia"
4,COL SAN JUAN VIANETH ...,KR 2 C 3 18 SUR,"KR 2 C 3 18 SUR,Bogota,Colombia","CARRERA 2 C 3 18 SUR,Bogota,Colombia"
...,...,...,...,...
2399,COLEGIO SAN JOSE DE CALASANZ,CL 129 88 B 35,"CL 129 88 B 35,Bogota,Colombia","CALLE 129 88 B 35,Bogota,Colombia"
2400,COL NUEVO REINO DE TURINGIA,KR 103 B 152 C 34,"KR 103 B 152 C 34,Bogota,Colombia","CARRERA 103 B 152 C 34,Bogota,Colombia"
2401,LIC INF GOOFY ...,CL 65 F SUR 78 C 32,"CL 65 F SUR 78 C 32,Bogota,Colombia","CALLE 65 F SUR 78 C 32,Bogota,Colombia"
2402,LIC APRENDO CON MIS AMIGOS ...,CR. 94 NO.29-30,"CR. 94 NO.29-30,Bogota,Colombia","CR. 94 NO.29-30,Bogota,Colombia"


In [20]:
df2['coordenada']=df2['direccion-corregida'].apply(obtenerCoordenadas)

C:\Users\1061708930\AppData\Local\Temp\ipykernel_14972\3427630671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['coordenada']=df2['direccion-corregida'].apply(obtenerCoordenadas)


In [21]:
df2

,nombreestablecimiento,direccion,direccion-completa,direccion-corregida,coordenada
0,INST DE BTO TEC COMERCIAL PITAGORAS ...,KR 5 11 67,"KR 5 11 67,Bogota,Colombia","CARRERA 5 11 67,Bogota,Colombia","(4.5977768, -74.07332459999999)"
1,GIMNASIO INFANTIL FEDERICO FROEBEL,CL 32 SUR 52 A 54,"CL 32 SUR 52 A 54,Bogota,Colombia","CALLE 32 SUR 52 A 54,Bogota,Colombia","(4.6050717, -74.12871609999999)"
2,GIMNASIO INFANTIL LOS NIÑOS DEL MAÑANA,CL 134 D 47 27,"CL 134 D 47 27,Bogota,Colombia","CALLE 134 D 47 27,Bogota,Colombia","(4.7210207, -74.05454519999999)"
3,COLEGIO FRANCISCO PRIMERO S.S. (IED),KR 28 63 64,"KR 28 63 64,Bogota,Colombia","CARRERA 28 63 64,Bogota,Colombia","(4.6521273, -74.0756332)"
4,COL SAN JUAN VIANETH ...,KR 2 C 3 18 SUR,"KR 2 C 3 18 SUR,Bogota,Colombia","CARRERA 2 C 3 18 SUR,Bogota,Colombia","(4.6161775, -74.06377429999999)"
...,...,...,...,...,...
2399,COLEGIO SAN JOSE DE CALASANZ,CL 129 88 B 35,"CL 129 88 B 35,Bogota,Colombia","CALLE 129 88 B 35,Bogota,Colombia","(4.7232102, -74.0883631)"
2400,COL NUEVO REINO DE TURINGIA,KR 103 B 152 C 34,"KR 103 B 152 C 34,Bogota,Colombia","CARRERA 103 B 152 C 34,Bogota,Colombia","(4.7530499, -74.0913225)"
2401,LIC INF GOOFY ...,CL 65 F SUR 78 C 32,"CL 65 F SUR 78 C 32,Bogota,Colombia","CALLE 65 F SUR 78 C 32,Bogota,Colombia","(4.6015631, -74.18503749999999)"
2402,LIC APRENDO CON MIS AMIGOS ...,CR. 94 NO.29-30,"CR. 94 NO.29-30,Bogota,Colombia","CR. 94 NO.29-30,Bogota,Colombia","(4.6708318, -74.1380677)"


In [22]:
df2.to_csv("coordenadas.csv")